In [1]:
import os
from PIL import Image
import xml.etree.ElementTree as ET
import numpy as np

In [2]:
train_label_xml_path = './datasets/taydo_splitted_data/labels/train_xml/'
test_label_xml_path = './datasets/taydo_splitted_data/labels/test_xml/'
validation_label_xml_path = './datasets/taydo_splitted_data/labels/val_xml/'

train_dots_path = './datasets/taydo_splitted_data/train/coordinate_texts/'
test_dots_path = './datasets/taydo_splitted_data/test/coordinate_texts/'
validation_dots_path = './datasets/taydo_splitted_data/validation/coordinate_texts/'

train_masks_path = './datasets/taydo_splitted_data/train/dots/'
test_masks_path = './datasets/taydo_splitted_data/test/dots/'
validation_masks_path = './datasets/taydo_splitted_data/validation/dots/'

In [3]:
img = Image.open('./splitted_data/examples/image8004.jpg').convert('L')

In [4]:
pix = np.array(img)

In [5]:
pix.shape

(1080, 1920)

In [6]:
pix

array([[116, 116, 116, ..., 142, 138, 137],
       [116, 116, 116, ..., 141, 139, 138],
       [116, 116, 116, ..., 137, 135, 135],
       ...,
       [ 84,  85,  85, ...,  78,  78,  78],
       [ 32,  33,  33, ...,  32,  32,  32],
       [  3,   4,   5, ...,   4,   4,   4]], dtype=uint8)

In [7]:
init_img = np.zeros(pix.shape, dtype='uint8')

In [8]:
init_img.shape

(1080, 1920)

In [31]:
def calc_centeroid_point(point_list):
    x1 = point_list[0]
    y1 = point_list[1]
    x2 = point_list[2]
    y2 = point_list[3]
    centeroid = [(x1 + x2) // 2, (y1 + y2) // 2]

    return centeroid


def get_bounding_box_points(path):
    bounding_box_collections = []
    for file_xml in os.listdir(path):
        info = {
            'file_name': file_xml,
            'bounding_box_points': None,
            'image_info': {
                'height': None,
                'width': None,
                'channel': None
            },
        }
        bouding_boxes_xml_path = os.path.join(path, file_xml)

        tree = ET.parse(bouding_boxes_xml_path)
        root = tree.getroot()

        bounding_box_list = []
        for child in root:
            if child.tag == 'size':
                for size_child in child:
                    if size_child.tag == 'width' or size_child.tag == 'height':
                        info['image_info'][size_child.tag] = size_child.text
                    if size_child.tag == 'depth':
                        info['image_info']['channel'] = size_child.text

            if child.tag == 'object':
                for obj_child in child:
                    if obj_child.tag == 'bndbox':
                        x_min = int(obj_child[0].text)
                        y_min = int(obj_child[1].text)
                        x_max = int(obj_child[2].text)
                        y_max = int(obj_child[3].text)
                        bounding_box_list.append([x_min, y_min, x_max, y_max])
        info['bounding_box_points'] = bounding_box_list

        bounding_box_collections.append(info)

    return bounding_box_collections


def save_bounding_box_to_dots(save_coordinate_directory_path, save_image_mask_directory_path, bounding_box_collections):
    for info in bounding_box_collections:
        image_width = int(info['image_info']['width'])
        image_height = int(info['image_info']['height'])
        image_array = np.zeros((image_height, image_width), dtype='uint8')

        saved_file = os.path.splitext(save_coordinate_directory_path + info['file_name'])[0] + '.txt'
        saved_image_file = os.path.splitext(save_image_mask_directory_path + info['file_name'])[0] + '.png'
        f = open(saved_file, "w")
        for points in info['bounding_box_points']:
            centeroid = calc_centeroid_point(points)
            image_array[centeroid[1],centeroid[0]] = 255
            image = Image.fromarray(image_array)
            image.save(saved_image_file)
            f.write('\t'.join([str(centeroid[0]), str(centeroid[1])]) + '\n')
        f.close()

In [32]:
bounding_box_collections = get_bounding_box_points(train_label_xml_path)
save_bounding_box_to_dots(train_dots_path, train_masks_path, bounding_box_collections)

In [26]:
len(bounding_box_collections)

10

In [27]:
bounding_box_collections

[{'file_name': 'image7995.xml',
  'bounding_box_points': [[689, 402, 829, 542],
   [920, 348, 1145, 452],
   [1765, 372, 1920, 684],
   [785, 374, 838, 457],
   [703, 368, 757, 413],
   [893, 336, 930, 384]],
  'image_info': {'height': '1080', 'width': '1920', 'channel': '3'}},
 {'file_name': 'image8001.xml',
  'bounding_box_points': [[691, 405, 837, 548],
   [980, 324, 1238, 502],
   [777, 351, 842, 409],
   [1577, 428, 1810, 778],
   [1400, 318, 1530, 529],
   [839, 352, 883, 425]],
  'image_info': {'height': '1080', 'width': '1920', 'channel': '3'}},
 {'file_name': 'image7994.xml',
  'bounding_box_points': [[693, 405, 842, 540],
   [899, 348, 1122, 455],
   [847, 347, 888, 407],
   [772, 364, 823, 433]],
  'image_info': {'height': '1080', 'width': '1920', 'channel': '3'}},
 {'file_name': 'image7993.xml',
  'bounding_box_points': [[688, 407, 831, 553],
   [777, 369, 999, 468],
   [770, 360, 812, 414],
   [943, 337, 1000, 405]],
  'image_info': {'height': '1080', 'width': '1920', 'cha